In [11]:
import contextlib
from distutils.dir_util import copy_tree
import shutil
from os.path import join
from os import mkdir
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
from random import random

In [12]:
dataset_folder = "Original"
data_id = "INDEX"
fold_col = "INDEX_FOLDS"
target_col = "TARGET"
output_folder = "US_Corn_Yield_2016_Removed_"
root_path =  "/exp/tpinho/Datasets/US_Corn_Yield_2016"
data_path = f"/exp/tpinho/Datasets/US_Corn_Yield_2016/{dataset_folder}/data.csv"
meshblock_folder = join(root_path, dataset_folder, "meshblocks")

In [13]:
def _mkdir(dir, folder_name: str) -> None:  # sourcery skip: avoid-builtin-shadow
        """Creates a folder at current path"""
        dir = join(dir, folder_name)
        with contextlib.suppress(FileExistsError):
                mkdir(dir)
        return dir

regions = {"Northeast": ["Maine", "Massachusetts", "Rhode Island", "Connecticut", "New Hampshire", "Vermont", "New York", "Pennsylvania", "New Jersey", "Delaware", "Maryland"],
           "Southeast": ["West Virginia", "Virginia", "Kentucky", "Tennessee", "North Carolina", "South Carolina", "Georgia", "Alabama", "Mississippi", "Arkansas", "Louisiana", "Florida"],
           "Midwest": ["Ohio", "Indiana", "Michigan", "Illinois", "Missouri", "Wisconsin", "Minnesota", "Iowa", "Kansas", "Nebraska", "South Dakota", "North Dakota"],
           "Southwest": ["Texas", "Oklahoma", "New Mexico", "Arizona"],
           "West": ["Colorado", "Wyoming", "Montana", "Idaho", "Washington", "Oregon", "Utah", "Nevada", "California", "Alaska", "Hawaii"]
          }
for region in regions:
        regions[region] = [x.upper() for x in regions[region]]


In [14]:

data = pd.read_csv(join(root_path, dataset_folder, "data.csv"), index_col=data_id)


for region in tqdm(regions):
    output_path = _mkdir(root_path, output_folder + region)
    copy_tree(meshblock_folder, join(output_path,"meshblocks"))
    shutil.copyfile(join(root_path, dataset_folder, "normd_matrix.csv"), join(output_path, "normd_matrix.csv"))
    shutil.copyfile(join(root_path, dataset_folder, "queen_matrix.csv"), join(output_path, "queen_matrix.csv"))
    data_sampled = data[~data[fold_col].isin(regions[region])].copy()
    data_sampled.to_csv(join(output_path, "data.csv"), index=True)
    

100%|██████████| 5/5 [02:05<00:00, 25.17s/it]
